## my class

In [1]:
class Data():
    def __init__(self, data):
        self.data = data
        
    def __repr__(self):
        return repr(self.data)
    
    def filter(self, *keys):
        if len(keys) == 1:
            if keys[0] == "all":
                return Data(self.data)
            else:
                return Data({k:v for k, v in self.data.items() if k == keys[0]})
        else:
            temp = self.filter(keys[0])
            return Data({k : Data(v).filter(*keys[1:]) for k, v in temp.data.items()})


# Example usage
my_dict = {
    "key1": {"k1":"key1_v1", "key1_k2":"key1_v2"},
    "key2": {"k2":{"k1":"v1"}, "key2_k2":"key2_v2"},
    "key3": {"k2":"key3_v1", "key3_k2":"key3_v2"}
}

temp = Data(my_dict)

In [2]:
print(temp.filter("key1"))
print(temp.filter("all").filter("key2"))

{'key1': {'k1': 'key1_v1', 'key1_k2': 'key1_v2'}}
{'key2': {'k2': {'k1': 'v1'}, 'key2_k2': 'key2_v2'}}


In [27]:
print(temp.filter("key1","k1")) 

{'key1': {'k1': 'key1_v1'}}


In [28]:
print(temp.filter("key2","k2","k1")) 

{'key2': {'k2': {'k1': 'v1'}}}


## subclass from dict

In [1]:

class dData(dict):
    def __new__(cls, base = None):
        # print(cls)
        # print(base)
        if isinstance(base, dict):
            return super().__new__(cls)
        return base

    def cleanProps(self):
        if isinstance(self, dData):
            # temp = {k:dData(v).cleanProps() for k,v in self.items()}
            temp = {k:(dData(v).cleanProps() if isinstance(v, dict) else v) for k,v in self.items()}
            return dData({k:v for k,v in temp.items() if v != {}})

    def __repr__(self):
        return f'dData({super().__repr__()})'

    def filterKey(self, key):
        if key[0] == "all":
            return dData(self)
        elif isinstance(key, list):
            return dData({k:v for k, v in self.items() if k in key})
        else:
            return dData({k:v for k, v in self.items() if k == key})
    
    def filter(self, *keys):
        if len(keys) == 1:
            return dData(self.filterKey(keys[0]))
        else:
            firstKeyData = self.filterKey(keys[0])
            return dData({k : dData(v).filter(*keys[1:]) for k, v in firstKeyData.items()})
        
my_dict = {
    "key1": {"k1":"key1_v1", "key1_k2":"key2_v2"},
    "key2": {"k2":{"k1":"v1"}, "key2_k2":"key2_v2"},
    "key3": {"k2":"key3_v1", "key3_k2":"key3_v2"}
}

temp2 = dData(my_dict)

In [19]:
dData('key1_v1')

'key1_v1'

In [8]:
dData()

### base key

In [9]:
print(temp2.filterKey("notakey"))
print(temp2.filterKey("key1"))
print(temp2.filter("key1"))
print(temp2.filter("key2","k2","k3"))
print(temp2.filter("key2","notakey"))
temp2.items()

DictData({})
DictData({'key1': {'k1': 'key1_v1', 'key1_k2': 'key2_v2'}})
DictData({'key1': {'k1': 'key1_v1', 'key1_k2': 'key2_v2'}})
DictData({'key2': DictData({'k2': DictData({})})})
DictData({'key2': DictData({})})


dict_items([('key1', {'k1': 'key1_v1', 'key1_k2': 'key2_v2'}), ('key2', {'k2': {'k1': 'v1'}, 'key2_k2': 'key2_v2'}), ('key3', {'k2': 'key3_v1', 'key3_k2': 'key3_v2'})])

In [10]:
dData({'key2': {'k2': {}}}).cleanProps()

DictData({})

### list keys

In [3]:
print(temp2.filter(["key1", "key2"],"k1"))

DictData({'key1': DictData({'k1': 'key1_v1'}), 'key2': DictData({})})


In [4]:
dData({'key2': {'k2': {}}}).cleanProps()

DictData({})

In [5]:
temp2.filter(["key1", "key2"],"k1")

DictData({'key1': DictData({'k1': 'key1_v1'}), 'key2': DictData({})})

In [6]:
temp2.filter(["key1", "key2"],"k1").cleanProps()

DictData({'key1': DictData({'k1': 'key1_v1'})})

In [7]:
dData('key1_v1')

'key1_v1'

## pandas

In [33]:
import pandas as pd

In [34]:
my_dict = {
    "key1": {"k1":"key1_v1", "key1_k2":"key2_v2"},
    "key2": {"k2":{"k1":"v1"}, "key2_k2":"key2_v2"},
    "key3": {"k2":"key3_v1", "key3_k2":"key3_v2"}
}

df = pd.DataFrame(my_dict)

In [35]:
a = pd.DataFrame(my_dict)
b = pd.DataFrame.from_dict(my_dict)
a.equals(b)

True

In [36]:
df["key1"]

k1         key1_v1
key1_k2    key2_v2
k2             NaN
key2_k2        NaN
key3_k2        NaN
Name: key1, dtype: object

In [16]:
import pandas as pd
df = pd.DataFrame.from_dict(my_dict, orient='index')
df

,k1,key1_k2,k2,key2_k2,key3_k2
key1,key1_v1,key2_v2,NaN,NaN,NaN
key2,NaN,NaN,{'k1': 'v1'},key2_v2,NaN
key3,NaN,NaN,key3_v1,NaN,key3_v2


In [17]:
df.query("k1 == 'key1_v1'")

,k1,key1_k2,k2,key2_k2,key3_k2
key1,key1_v1,key2_v2,NaN,NaN,NaN


In [ ]:
filtered_df = df[df['age'] > 25].to_dict(orient='records') 

In [63]:
df['age'] > 25

0    False
1     True
2    False
Name: age, dtype: bool

In [71]:
df['age']

0              25
1              30
2    {'ab': 'dd'}
Name: age, dtype: object

In [64]:
print(df["color"])
print(df[df["color"]=="color"])

0    NaN
1    NaN
2    red
Name: color, dtype: object
Empty DataFrame
Columns: [name, age, color]
Index: []


In [66]:
df[df["color"]=="red"]

,name,age,color
2,Charlie,22,red


In [58]:
df2["key1"]

k1         key1_v1
key1_k2    key2_v2
k2             NaN
key2_k2        NaN
key3_k2        NaN
Name: key1, dtype: object

In [74]:
a, b = 1, 2
print(a)

1
